In [1]:
#default_exp ghtop

# ghtop API

> API details

In [2]:
#export
import sys, signal, shutil, os, json, emoji, enlighten
from dashing import *
from collections import defaultdict
from warnings import warn
from itertools import islice

from fastcore.utils import *
from fastcore.foundation import *
from fastcore.script import *
from ghapi.all import *
from ghtop.richext import *
from ghtop.all_rich import (Console, Color, FixedPanel, box, Segments, Live,
                            grid, ConsoleOptions, Progress, BarColumn, Spinner)

In [3]:
evts = load_sample_events()

In [4]:
#export
def get_title(nm): return camel2words(remove_suffix(nm,'Event'))
ETYPES = PushEvent,PullRequestEvent,IssueCommentEvent, ReleaseEvent

def get_sparklines(types=ETYPES):
    ecolors = {t.__name__: ifnone(colors.get(t.__name__, None), 'default') for t in types}
    return SparkGrp([SparkCfg(str(v), get_title(k)) for k,v in ecolors.items()] + [SparkCfg('magenta', 'All Events')],
                    span=5,
                    spn_lbl='/5s')

In [5]:
#export
term = Terminal()

tdim = L(os.popen('stty size', 'r').read().split())
if not tdim: theight,twidth = 15,15
else: theight,twidth = tdim.map(lambda x: max(int(x)-5, 15))

In [6]:
#export
def github_auth_device(wb='', n_polls=9999):
    "Authenticate with GitHub, polling up to `n_polls` times to wait for completion"
    auth = GhDeviceAuth()
    print(f"First copy your one-time code: {term.yellow}{auth.user_code}{term.normal}")
    print(f"Then visit {auth.verification_uri} in your browser, and paste the code when prompted.")
    if not wb: wb = input("Shall we try to open the link for you? [y/n] ")
    if wb[0].lower()=='y': auth.open_browser()

    print("Waiting for authorization...", end='')
    token = auth.wait(lambda: print('.', end=''), n_polls=n_polls)
    if not token: return print('Authentication not complete!')
    print("Authenticated with GitHub")
    return token

When we run this we'll be shown a URL to visit and a code to enter in order to authenticate. Normally we'll be prompted to open a browser, and the function will wait for authentication to complete -- for demonstrating here we'll skip both of these steps:

In [7]:
github_auth_device('n',n_polls=0)

First copy your one-time code: 36BE-1EC5
Then visit https://github.com/login/device in your browser, and paste the code when prompted.
Waiting for authorization...Authentication not complete!


In [8]:
#export
def _exit(msg):
    print(msg, file=sys.stderr)
    sys.exit()

In [9]:
#exports
def limit_cb(rem,quota):
    "Callback to warn user when close to using up hourly quota"
    w='WARNING '*7
    if rem < 1000: print(f"{w}\nRemaining calls: {rem} out of {quota}\n{w}", file=sys.stderr)

When creating `GhApi` we can pass a callback which will be called after each API operation. In this case, we use it to warn the user when their quota is getting low.

In [10]:
#export
Events = dict(
    IssuesEvent_closed=('⭐', 'closed', noop),
    IssuesEvent_opened=('📫', 'opened', noop),
    IssueCommentEvent=('💬', 'commented on', term.white),
    PullRequestEvent_opened=('✨', 'opened a pull request', term.yellow),
    PullRequestEvent_closed=('✔', 'closed a pull request', term.green),
)

In [11]:
#export
def _to_log(e):
    login,repo,pay = e.actor.login,e.repo.name,e.payload
    typ = e.type + (f'_{pay.action}' if e.type in ('PullRequestEvent','IssuesEvent') else '')
    emoji,msg,color = Events.get(typ, [0]*3)
    if emoji:
        xtra = '' if e.type == "PullRequestEvent" else f' issue # {pay.issue.number}'
        d = try_attrs(pay, "pull_request", "issue")
        return color(f'{emoji} {login} {msg}{xtra} on repo {repo[:20]} ("{d.title[:50]}...")')
    elif e.type == "ReleaseEvent": return f'🚀 {login} released {e.payload.release.tag_name} of {repo}'

In [12]:
#export
def print_event(e, counter):
    res = _to_log(e)
    if res: print(res)
    elif counter and e.type == "PushEvent": [counter.update() for c in e.payload.commits]
    elif e.type == "SecurityAdvisoryEvent": print(term.blink("SECURITY ADVISORY"))

We can pretty print a selection of event types using `print_event`, e.g:

In [13]:
for e in evts[:100]: print_event(e,None)

✔ Didier-D-crypto closed a pull request on repo Didier-D-crypto/Empl ("Bump ini from 1.3.5 to 1.3.8...")
💬 Holzhaus commented on issue # 168 on repo mixxxdj/website ("Site of mixxx is out...")
💬 reaperrr commented on issue # 18828 on repo OpenRA/OpenRA ("InvalidDataException: nuyell1.aud is not a valid s...")
✨ fantommohammed opened a pull request on repo fantommohammed/Mu-lt ("Begin...")
💬 0pdd commented on issue # 2 on repo proofit404/cruftbot ("Initialize Django project....")
✨ dependabot[bot] opened a pull request on repo Didier-D-crypto/Empl ("Bump dot-prop from 4.2.0 to 4.2.1...")
📫 oulasvirta opened issue # 11 on repo oulasvirta/write-git ("Title...")
✔ pull[bot] closed a pull request on repo sgrayban/github-read ("[pull] master from anuraghazra:master...")
✨ dseo3 opened a pull request on repo dseo3/GroupProject_T ("Populating data inside of bookmark...")
✨ snyk-bot opened a pull request on repo BlueCC8/iot-simulato ("[Snyk] Security upgrade mongoose from 5.4.11 to 5....")
✔ re

In [14]:
#export
def pct_comp(api): return int(((5000-int(api.limit_rem)) / 5000) * 100)

In [15]:
#export
def tail_events(evt, api):
    "Print events from `fetch_events` along with a counter of push events"
    p = FixedPanel(theight, box=box.HORIZONTALS, title='ghtop')
    s = get_sparklines()
    g = grid([[s], [p]])
    with Live(g):
        for e in evt:
            k = get_title(e.type)
            if k in s.sparklines: s.update({'All Events':1, k: 1}, pct_complete=pct_comp(api))
            else: s.update({'All Events':1}, pct_complete=pct_comp(api))
            p.append(e)
            g = grid([[s], [p]])

In [16]:
#export
def _pr_row(*its): print(f"{its[0]: <30} {its[1]: <6} {its[2]: <5} {its[3]: <6} {its[4]: <7}")
def watch_users(evts):
    "Print a table of the users with the most events"
    users,users_events = defaultdict(int),defaultdict(lambda: defaultdict(int))
    while True:
        for x in islice(evts, 10):
            users[x.actor.login] += 1
            users_events[x.actor.login][x.type] += 1

        print(term.clear())
        _pr_row("User", "Events", "PRs", "Issues", "Pushes")
        sorted_users = sorted(users.items(), key=lambda o: (o[1],o[0]), reverse=True)
        for u in sorted_users[:20]:
            _pr_row(*u, *itemgetter('PullRequestEvent','IssuesEvent','PushEvent')(users_events[u[0]]))

In [17]:
#export
def _push_to_log(e): return f"{e.actor.login} pushed {len(e.payload.commits)} commits to repo {e.repo.name}"
def _logwin(title,color): return Log(title=title,border_color=2,color=color)

# def quad_logs(evts, api):
#     "Print 4 panels, showing most recent issues, commits, PRs, and releases"
#     print(len(evts))
#     term.enter_fullscreen()
#     ui = HSplit(VSplit(_logwin('Issues',        color=7), _logwin('Commits' , color=3)),
#                 VSplit(_logwin('Pull Requests', color=4), _logwin('Releases', color=5)))

#     issues,commits,prs,releases = all_items = ui.items[0].items+ui.items[1].items
#     for o in all_items: o.append(" ")

#     d = dict(PushEvent=commits, IssuesEvent=issues, IssueCommentEvent=issues, PullRequestEvent=prs, ReleaseEvent=releases)
#     while True:
#         for x in islice(evts, 10):
#             f = [_to_log,_push_to_log][x.type == 'PushEvent']
#             if x.type in d: d[x.type].append(f(x)[:95])
#         ui.display()

In [18]:
    ps = {o:FixedPanel(height=theight//2,
                       width=twidth//2, 
                       box=box.HORIZONTALS, 
                       title=camel2words(remove_suffix(o.__name__,'Event'))) for o in ETYPES}

In [19]:
#export
def _panelDict2Grid(pd):
    ispush,ispr,isiss,isrel = pd.values()
    return grid([[ispush,ispr],[isiss,isrel]], width=twidth)


def quad_logs(evts, api):
    "Print 4 panels, showing most recent issues, commits, PRs, and releases"
    pd = {o:FixedPanel(height=(theight//2)-2,
                       width=(twidth//2)-2, 
                       box=box.HORIZONTALS, 
                       title=camel2words(remove_suffix(o.__name__,'Event'))) for o in ETYPES}
    p = _panelDict2Grid(pd)
    s = get_sparklines()
    g = grid([[s], [p]])
    with Live(g):
        for e in evts:
            k = get_title(e.type)
            if k in s.sparklines: s.update({'All Events':1, k: 1}, pct_complete=pct_comp(api))
            else: s.update({'All Events':1}, pct_complete=pct_comp(api))
            typ = type(e)
            if typ in pd: pd[typ].append(e)
            p = _panelDict2Grid(pd)
            g = grid([[s], [p]])

In [20]:
#export
def simple(evts):
    for ev in evts: print(f"{ev.actor.login} {ev.type} {ev.repo.name}")

In [21]:
#export
def _get_token():
    path = Path.home()/".ghtop_token"
    if path.is_file():
        try: return path.read_text().strip()
        except: _exit("Error reading token")
    else: token = github_auth_device()
    path.write_text(token)
    return token

In [22]:
#export
def _signal_handler(sig, frame):
    if sig != signal.SIGINT: return
    print(term.exit_fullscreen(),term.clear(),term.normal)
    sys.exit(0)

_funcs = dict(tail=tail_events, quad=quad_logs, users=watch_users, simple=simple)
_filts = str_enum('_filts', 'user', 'repo', 'org')
_OpModes = str_enum('_OpModes', *_funcs)

@call_parse
def main(mode:         Param("Operation mode to run", _OpModes),
         include_bots: Param("Include bots (there's a lot of them!)", store_true)=False,
         types:        Param("Comma-separated types of event to include (e.g PushEvent)", str)='',
         filt:         Param("Filtering method", _filts)=None,
         filtval:      Param("Value to filter by (for `repo` use format `owner/repo`)", str)=None):
    signal.signal(signal.SIGINT, _signal_handler)
    types = types.split(',') if types else None
    if filt and not filtval: _exit("Must pass `filter_value` if passing `filter_type`")
    if filtval and not filt: _exit("Must pass `filter_type` if passing `filter_value`")
    kwargs = {filt:filtval} if filt else {}
    api = GhApi(limit_cb=limit_cb, token=_get_token())
    evts = api.fetch_events(types=types, incl_bot=include_bots, **kwargs)
    _funcs[mode](evts, api)

## Export -

In [23]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_ghtop.ipynb.
Converted Prototype_Rich.ipynb.
Converted index.ipynb.
Converted richext.ipynb.


In [24]:
api=GhApi()
revts = api.fetch_events()

In [25]:
# from collections import Counter

# lst = []
# counter = 0
# for x in revts:
#     lst.append(x)
#     counter+=1
#     if counter > 500: break
        
# Counter([x.type for x in lst]).most_common()